In [ ]:
# Install the package
!pip install easy_es

In [1]:
import pandas as pd

from easy_es import EventStudy, plot_mean_car, calculate_car_stats, plot_joint_mean_car

### Run single event study

In [2]:
# Load test events
events_df = pd.read_csv('https://raw.githubusercontent.com/Darenar/easy-event-study/main/tests/test_one/input_events.csv')

In [3]:
# Initialize EventStudy class object
event_study = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5',
    n_cores=1 # Change for multiprocessing
)

# Run the study
results_df = event_study.run_study(events_df)

100%|██████████| 33/33 [00:18<00:00,  1.74it/s]


In [4]:
# Calculate average results in a dataframe format
calculate_car_stats(results_df, critical_value=0.95)

,mean,median,sd,t_stat,p_value,upper_ci,lower_ci
Offset,,,,,,,
-10,-0.005119,-0.003075,0.002696,-1.898445,0.058,0.000166,-0.010403
-9,-0.006913,-0.005001,0.003902,-1.771815,0.076,0.000734,-0.014561
-8,-0.008918,-0.003972,0.006029,-1.479155,0.139,0.002899,-0.020736
-7,-0.009972,-0.004459,0.006938,-1.437230,0.151,0.003627,-0.023571
-6,-0.014504,-0.002461,0.007390,-1.962643,0.050,-0.000020,-0.028989
-5,-0.008355,0.002768,0.008282,-1.008808,0.313,0.007877,-0.024587
-4,-0.008015,0.002879,0.009823,-0.815919,0.415,0.011238,-0.027267
-3,-0.008249,0.003756,0.010280,-0.802499,0.422,0.011898,-0.028397
-2,-0.005908,-0.001566,0.010336,-0.571540,0.568,0.014351,-0.026167


In [5]:
# Plot graph with the results
plot_mean_car(results_df, critical_value=0.9)

### Pre-loading returns data

In [6]:
### One can explicitely load the returns data and to dump into a CSV file 
event_study.add_returns(
    list_of_tickers=events_df['ticker'].unique(),
    min_date=events_df['event_date'].min(),
    max_date=events_df['event_date'].max()
)
event_study.returns_df.to_csv('returns_test_dump.csv', index=False)

100%|██████████| 33/33 [00:19<00:00,  1.68it/s]


In [7]:
# Then to reload it and initialize the event-study with pre-loaded returns
event_study = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5'
)
event_study.add_returns(ret_df=pd.read_csv('returns_test_dump.csv'))

# Run the study
results_df = event_study.run_study(events_df)

### Plotting two event study results together

In [8]:
# Run first event study
events_df_one = pd.read_csv('https://raw.githubusercontent.com/Darenar/easy-event-study/main/tests/test_one/input_events.csv')
event_study_one = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5'
)
result_one = event_study_one.run_study(events_df_one)

100%|██████████| 33/33 [00:19<00:00,  1.71it/s]


In [9]:
# Run second event study
events_df_two = pd.read_csv('https://raw.githubusercontent.com/Darenar/easy-event-study/main/tests/test_two/input_events.csv')
event_study_two = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5'
    
)
result_two = event_study_two.run_study(events_df_two)

100%|██████████| 41/41 [00:23<00:00,  1.73it/s]


In [10]:
plot_joint_mean_car(
    result_one, 
    result_two, 
    name_one='Increase', 
    name_two='Decrease')

### Bootstrap test

The confidence intervals around mean CAR estimates are calculated using the parametric method.

One also could calculate them using non-parametric approach - by bootstrap. Strategy:
1. Given a list of ticker and events, randomly permute the events N times.
2. Calculate mean CAR estimate for each of these N-runs.
3. Calculate confidence by using quantiles (e.g. 0.05 and 0.95) on those N runs.
4. Plot together with the original mean CAR to see the significance.


Note: highly depends on the number of bootstrap runs. If possible, increase **n_cores** parameter to the greater number of cpu in use

In [11]:
# Initialize EventStudy class object
event_study = EventStudy(
    estimation_days=255,
    gap_days=50,
    window_after=10,
    window_before=10,
    min_estimation_days=100,
    estimator_type='ff5',
    n_cores=10 # Better to set up to higher number
)

# Run the study
results_df = event_study.run_study(events_df)

100%|██████████| 33/33 [00:19<00:00,  1.67it/s]


In [12]:
bootstrap_results_df = event_study.run_bootstrap_study(events_df, n_bootstrap=1000)

100%|██████████| 1000/1000 [00:00<00:00, 4003.78it/s]


In [13]:
# Each column represents mean CAR value obtained in each bootstrap run
bootstrap_results_df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
Offset,,,,,,,,,,,,,,,,,,,,,
-10,0.003908,0.001958,0.001706,-0.000306,-0.001315,-0.004910,-0.001511,0.004605,-0.001010,0.000974,...,0.002674,0.002309,-0.000555,-0.001330,0.001536,0.005051,0.000318,0.003598,0.003169,0.000637
-9,0.000139,-0.001565,0.003405,-0.004681,-0.000206,-0.000626,-0.005096,0.007760,0.004265,0.003993,...,0.002301,-0.001111,0.005391,-0.002488,0.002864,0.008674,-0.003128,0.004204,0.007928,-0.002167
-8,0.002001,-0.003181,0.001852,-0.004482,-0.002202,-0.005423,-0.006456,0.005994,0.006729,0.001809,...,0.001754,-0.000771,-0.006833,-0.002725,0.002493,0.011718,-0.006527,0.000417,0.010843,-0.002920
-7,0.003449,-0.002380,0.001628,-0.007611,0.001026,-0.000579,-0.007879,0.010209,0.006446,0.003416,...,0.003848,0.002868,-0.005962,0.000425,0.001624,0.009408,-0.011973,-0.003674,0.009949,0.000645
-6,0.003280,-0.008792,0.000061,-0.002807,0.006173,-0.002020,-0.009113,0.015390,0.003520,0.000965,...,0.002808,0.009904,-0.004273,0.005523,0.000714,0.012518,-0.014787,-0.003044,0.010158,0.002155
-5,0.004053,-0.008165,0.002873,-0.004620,0.004904,-0.007151,-0.007246,0.011030,-0.002442,0.001658,...,0.004340,0.008086,-0.005490,0.008644,0.000890,0.011375,-0.013593,-0.006297,0.010708,0.005012
-4,0.006434,-0.008377,-0.000080,-0.010525,0.002295,-0.011912,-0.009854,0.011708,-0.004205,0.000249,...,-0.002447,0.010993,-0.008822,0.004358,-0.003935,0.010620,-0.013795,-0.003950,0.013445,0.007532
-3,0.010246,-0.002298,-0.007039,-0.013668,0.006914,-0.006778,-0.006888,0.008620,-0.004121,0.001943,...,-0.001093,0.008421,-0.003713,0.004867,-0.003413,0.009557,-0.012470,0.001424,0.012858,0.007168
-2,0.013178,-0.003237,-0.009566,-0.014130,0.006864,0.000482,-0.004884,0.008876,-0.004265,0.009852,...,-0.004111,0.009496,-0.001138,0.006265,0.001493,0.006243,-0.014722,-0.001426,0.019716,0.009291


In [14]:
# Provide plotting function with a calculated bootstrap results to plot on the graph
plot_mean_car(results_df, bootstrap_df=bootstrap_results_df, critical_value=0.99)